In [2]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

import sys
sys.path.append("..")

In [3]:
cust = pd.read_csv("train-item-views.csv",delimiter=";")
cust

,sessionId,userId,itemId,timeframe,eventdate
0,1,NaN,81766,526309,2016-05-09
1,1,NaN,31331,1031018,2016-05-09
2,1,NaN,32118,243569,2016-05-09
3,1,NaN,9654,75848,2016-05-09
4,1,NaN,32627,1112408,2016-05-09
...,...,...,...,...,...
1235375,600684,NaN,42906,632853,2016-04-14
1235376,600684,NaN,33312,643522,2016-04-14
1235377,600684,NaN,33312,52621,2016-04-14
1235378,600684,NaN,5227,575276,2016-04-14


In [4]:
cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1235380 entries, 0 to 1235379
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   sessionId  1235380 non-null  int64  
 1   userId     372991 non-null   float64
 2   itemId     1235380 non-null  int64  
 3   timeframe  1235380 non-null  int64  
 4   eventdate  1235380 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 47.1+ MB


In [5]:
cust_final = cust.drop(columns = ['sessionId', 'itemId', 'timeframe', 'eventdate'])

In [6]:
print(cust_final.shape)

(1235380, 1)


In [7]:
cust_final.dropna()

,userId
164,2.0
165,2.0
166,2.0
167,2.0
168,2.0
...,...
1235338,249327.0
1235358,249344.0
1235359,249344.0
1235360,249344.0


In [8]:
train_queries = pd.read_csv("train-queries.csv", sep=";")
train_queries

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,queryId,sessionId,userId,timeframe,duration,eventdate,searchstring.tokens,categoryId,items,is.test
0,1,1,NaN,16327074,311,2016-05-09,"16655,244087,51531,529597,58153",0,"7518,71,30311,7837,30792,8252,81766,9338,62220...",False
1,2,2,NaN,705527,314,2016-05-09,"528941,529116",0,"70095,15964,8627,134850,32754,100747,74771,314...",False
2,3,3,NaN,0,502,2016-05-09,"133713,16655,138389",0,"59081,51125,9338,9550,32087,62793,2717,10403,3...",True
3,4,4,NaN,0,1092,2016-05-09,"3918,3822,460416,528812,5276,529517,528738",0,"46632,57465,79064,57748,6080,35997,47088,6078,...",False
4,5,5,NaN,102700,266,2016-05-09,"529223,199482",0,"27312,84626,12621,46209,5140,57539,5368,12923,...",False
...,...,...,...,...,...,...,...,...,...,...
923122,980499,600685,NaN,0,407,2016-05-11,NaN,1174,"37197,37877,37878,30493,59914,249161,70728,600...",True
923123,980500,13265,NaN,361805,1203,2016-04-04,NaN,257,"16285,383058,391318,334708,50753,39983,112937,...",False
923124,980501,286379,NaN,953967,3789,2016-04-07,NaN,273,"93417,38387,81656,161236,81657,212080,152555,1...",True
923125,980502,600686,249346.0,0,1581,2016-05-15,NaN,965,"7516,30087,7553,37380,30261,59236,48347,261,20...",True


In [9]:
train_queries.drop(columns = ['queryId', 'sessionId', 'timeframe', 'duration', 'eventdate', 'searchstring.tokens','categoryId', 'is.test'])

,userId,items
0,NaN,"7518,71,30311,7837,30792,8252,81766,9338,62220..."
1,NaN,"70095,15964,8627,134850,32754,100747,74771,314..."
2,NaN,"59081,51125,9338,9550,32087,62793,2717,10403,3..."
3,NaN,"46632,57465,79064,57748,6080,35997,47088,6078,..."
4,NaN,"27312,84626,12621,46209,5140,57539,5368,12923,..."
...,...,...
923122,NaN,"37197,37877,37878,30493,59914,249161,70728,600..."
923123,NaN,"16285,383058,391318,334708,50753,39983,112937,..."
923124,NaN,"93417,38387,81656,161236,81657,212080,152555,1..."
923125,249346.0,"7516,30087,7553,37380,30261,59236,48347,261,20..."


In [10]:
train_f= train_queries.drop(columns = ['queryId', 'sessionId', 'timeframe', 'duration', 'eventdate', 'searchstring.tokens','categoryId', 'is.test'])
train_f

,userId,items
0,NaN,"7518,71,30311,7837,30792,8252,81766,9338,62220..."
1,NaN,"70095,15964,8627,134850,32754,100747,74771,314..."
2,NaN,"59081,51125,9338,9550,32087,62793,2717,10403,3..."
3,NaN,"46632,57465,79064,57748,6080,35997,47088,6078,..."
4,NaN,"27312,84626,12621,46209,5140,57539,5368,12923,..."
...,...,...
923122,NaN,"37197,37877,37878,30493,59914,249161,70728,600..."
923123,NaN,"16285,383058,391318,334708,50753,39983,112937,..."
923124,NaN,"93417,38387,81656,161236,81657,212080,152555,1..."
923125,249346.0,"7516,30087,7553,37380,30261,59236,48347,261,20..."


In [11]:
final_table=train_f.dropna()
final_table[:200]
final_table['userId'] = final_table['userId'].astype("int64")
final_table

/var/folders/k5/4j1l0hjx1cz26q703gh4y1kh0000gn/T/ipykernel_35515/3012063681.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['userId'] = final_table['userId'].astype("Int64")


,userId,items
47,2,"35389,68366,35438,5472,5683,24764,35627,46907,..."
59,3,"21100,113639,22128,2444,52938,129975,17247,329..."
104,4,"48336,30308,7763,7776,30839,81819,111066,8525,..."
107,5,"666,38247,8138,8619,32979,85571,33070,85771,10..."
129,6,"7837,513,30792,8252,416192,94453,293921,62239,..."
...,...,...
923114,62039,"26006,30180,48359,283,30313,7827,460,504,48768..."
923117,74686,"91036,37156,60,61,91096,7589,7590,110,58958,30..."
923118,189994,"59134,69919,59135,59248,7735,108619,20776,7006..."
923125,249346,"7516,30087,7553,37380,30261,59236,48347,261,20..."


In [12]:
#final_table['items']=final_table['items'].tolist()
#final_table

In [13]:
final_table['items'] = final_table['items'].apply(lambda x: [int(i) for i in x.split(',')])
final_table.head(2).set_index('userId')['items'].apply(pd.Series).reset_index()

/var/folders/k5/4j1l0hjx1cz26q703gh4y1kh0000gn/T/ipykernel_35515/4265787848.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_table['items'] = final_table['items'].apply(lambda x: [int(i) for i in x.split(',')])


,userId,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,2,35389,68366,35438,5472,5683,24764,35627,46907,13326,...,36031,36146,6308,6526,36411,36412,36410,6530,6560,6848
1,3,21100,113639,22128,2444,52938,129975,17247,32902,64097,...,43228,65293,3935,4699,67991,130748,6190,47159,25610,14635


In [14]:
import pandas as pd

data = pd.melt(final_table.set_index('userId')['items'].apply(pd.Series).reset_index(), 
             id_vars=['userId'],
             value_name='items') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['userId', 'items']) \
    .agg({'items': 'count'}) \
    .rename(columns={'items': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'items': 'itemId'})
data['itemId'] = data['itemId']

In [15]:
print(data.shape)
data.head()

(38013048, 3)


,userId,itemId,purchase_count
0,2,5472.0,1
1,2,5683.0,1
2,2,6308.0,1
3,2,6526.0,1
4,2,6530.0,1


In [16]:
data['itemId'] = data['itemId'].astype("int64")
print(data.shape)
data.head()


(38013048, 3)


,userId,itemId,purchase_count
0,2,5472,1
1,2,5683,1
2,2,6308,1
3,2,6526,1
4,2,6530,1


In [17]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy


In [18]:
data_dummy = create_data_dummy(data)

In [19]:
import pandas as pd
df_matrix = pd.pivot_table(data[:6000], values='purchase_count', index='userId', columns='itemId')
df_matrix.head()


itemId,43,58,59,63,67,71,157,163,176,218,...,597181,608474,611064,611115,613078,619574,625661,672648,678608,685992
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,NaN,...,1.0,1.0,1.0,1.0,NaN,1.0,1.0,1.0,1.0,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_matrix.shape

(56, 5383)

In [21]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
print(df_matrix_norm.shape)
df_matrix_norm.head()

(56, 5383)


itemId,43,58,59,63,67,71,157,163,176,218,...,597181,608474,611064,611115,613078,619574,625661,672648,678608,685992
userId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['userId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(638, 3)


,userId,itemId,scaled_purchase_freq
284,6,71,0.0
288,10,71,1.0
292,14,71,0.0
1180,6,513,1.0
1217,43,513,0.0


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38013048 entries, 0 to 38013047
Data columns (total 3 columns):
 #   Column          Dtype
---  ------          -----
 0   userId          int64
 1   itemId          int64
 2   purchase_count  int64
dtypes: int64(3)
memory usage: 870.0 MB


In [24]:
def split_data(data):
    train, test = train_test_split(data, test_size = .2)
    train = tc.SFrame(train)
    test = tc.SFrame(test)
    return train, test
# train_data = tc.SFrame(train)
# test_data = tc.SFrame(test)
# return train_data, test_data
# train_data, test_data = split_data(data)
# train_data_dummy, test_data_dummy = split_data(data_dummy)
# train_data_norm, test_data_norm = split_data(data)
# train_data_norm, test_data_norm = split_data(data_norm)

In [25]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [26]:
# variables to define field names
user_id = 'userId'
item_id = 'itemId'
target = 'purchase_count'
users_to_recommend = list(cust[user_id].dropna())
n_rec = 10 # number of items to recommend
n_display = 30

In [27]:
if(tc.load_model('popularity_model')):
    popularity_model = tc.load_model('popularity_model')
else:
    popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    popularity_model.save('popularity_model')

Preparing data set.

Data has 30410438 observations with 232643 users and 124602 items.

Data prepared in: 10.2343s

30410438 observations to process; with 124602 unique items.

In [41]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/372991 queries. users per second: 6297.74

recommendations finished on 2000/372991 queries. users per second: 6027.31

recommendations finished on 3000/372991 queries. users per second: 5838.43

recommendations finished on 4000/372991 queries. users per second: 5887.25

recommendations finished on 5000/372991 queries. users per second: 5876.13

...

recommendations finished on 368000/372991 queries. users per second: 5711.2

recommendations finished on 369000/372991 queries. users per second: 5699.24

recommendations finished on 370000/372991 queries. users per second: 5687.98

recommendations finished on 371000/372991 queries. users per second: 5670.92

recommendations finished on 372000/372991 queries. users per second: 5625.14

In [30]:
popularity_recomm.print_rows(n_display)

+--------+--------+-------------------+------+
| userId | itemId |       score       | rank |
+--------+--------+-------------------+------+
|   2    | 246644 |        14.0       |  1   |
|   2    | 477342 |        12.0       |  2   |
|   2    | 87111  |        12.0       |  3   |
|   2    | 352198 |        12.0       |  4   |
|   2    | 171183 |        9.0        |  5   |
|   2    | 414749 |        8.0        |  6   |
|   2    | 50863  | 6.833333333333333 |  7   |
|   2    | 257054 |        6.5        |  8   |
|   2    | 47504  |        6.0        |  9   |
|   2    | 14970  |        6.0        |  10  |
|   2    | 246644 |        14.0       |  1   |
|   2    | 477342 |        12.0       |  2   |
|   2    | 87111  |        12.0       |  3   |
|   2    | 352198 |        12.0       |  4   |
|   2    | 171183 |        9.0        |  5   |
|   2    | 414749 |        8.0        |  6   |
|   2    | 50863  | 6.833333333333333 |  7   |
|   2    | 257054 |        6.5        |  8   |
|   2    | 47

In [31]:
def model(train_data,user_id, item_id, target, users_to_recommend, n_rec, n_display):
    model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [32]:
if(tc.load_model('cos')):
    cos = tc.load_model('cos')
else:
    target = 'purchase_count'
    cos = model(train_data, user_id, item_id, target, users_to_recommend, n_rec, n_display)
    cos.save('cos_model')

Preparing data set.

Data has 30410438 observations with 232643 users and 124602 items.

Data prepared in: 10.9678s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 369000/372991 queries. users per second: 4696.57

recommendations finished on 370000/372991 queries. users per second: 4695.56

recommendations finished on 371000/372991 queries. users per second: 4693.97

recommendations finished on 372000/372991 queries. users per second: 4683.98

+--------+--------+---------------------+------+
| userId | itemId |        score        | rank |
+--------+--------+---------------------+------+
|   2    |  5643  |  0.5245915333429972 |  1   |
|   2    | 14228  |  0.5201462507247925 |  2   |
|   2    | 14001  |  0.4848324378331502 |  3   |
|   2    |  5828  | 0.48367162148157755 |  4   |
|   2    | 43616  | 0.48262713352839154 |  5   |
|   2    | 79520  | 0.48132483959197997 |  6   |
|   2    | 58054  | 0.48124775489171345 |  7   |
|   2    | 79341  |  0.4807570815086365 |  8   |
|   2    | 35598  | 0.47835321029027306 |  9   |
|   2    | 47070  | 0.47774721384048463 |  10  |
|   2    |  5643  |  0.5245915333429972 |  1   |
|   2    | 14228  |  0.5201462507247925 |  2   |
|   2    | 14001  |  0.4848324378331502 |  3   |
|   2    |  5828  | 0.48367162148157755 |  4   |
|   2    | 43616  | 0.48262713352839154 |  5   |
|   2    | 79520  | 0.48132483959197997 |  6   |
|   2    | 58054  | 0.48124775489171345 |  7   |
|   2    | 79341  | 

In [34]:
if(tc.load_model('cos_dummy')):
    cos_dummy = tc.load_model('cos_dummy')
else:
    target = 'purchase_dummy'
    cos_dummy = model(train_data_dummy, user_id, item_id, target, users_to_recommend, n_rec, n_display)
    cos_dummy.save('cos_dummy_model')

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 30410438 observations with 232648 users and 124548 items.

Data prepared in: 10.3508s

...

recommendations finished on 369000/372991 queries. users per second: 4365.76

recommendations finished on 370000/372991 queries. users per second: 4365.52

recommendations finished on 371000/372991 queries. users per second: 4364.63

recommendations finished on 372000/372991 queries. users per second: 4362.98

+--------+--------+---------------------+------+
| userId | itemId |        score        | rank |
+--------+--------+---------------------+------+
|   2    | 35010  | 0.44767352512904574 |  1   |
|   2    |  5828  |  0.4475716395037515 |  2   |
|   2    | 36411  | 0.44739533747945515 |  3   |
|   2    | 14228  |  0.4472952059337071 |  4   |
|   2    | 14446  |  0.4472178816795349 |  5   |
|   2    | 36266  |  0.4466805841241564 |  6   |
|   2    | 12086  |  0.4464670036520277 |  7   |
|   2    | 36082  | 0.44558458243097576 |  8   |
|   2    | 36271  |  0.4422305311475481 |  9   |
|   2    | 36363  |  0.4039241245814732 |  10  |
|   2    | 35010  | 0.44767352512904574 |  1   |
|   2    |  5828  |  0.4475716395037515 |  2   |
|   2    | 36411  | 0.44739533747945515 |  3   |
|   2    | 14228  |  0.4472952059337071 |  4   |
|   2    | 14446  |  0.4472178816795349 |  5   |
|   2    | 36266  |  0.4466805841241564 |  6   |
|   2    | 12086  |  0.4464670036520277 |  7   |
|   2    | 36082  | 

In [36]:
if(tc.load_model('cos_norm')):
    cos_norm = tc.load_model('cos_norm')
else:
    target = 'scaled_purchase_freq'
    cos_norm = model(train_data_norm, user_id, item_id, target, users_to_recommend, n_rec, n_display)
    cos_norm.save('cos_norm_model')

Preparing data set.

Data has 510 observations with 42 users and 277 items.

Data prepared in: 0.002493s

Training model from provided data.

Gathering per-item and per-user statistics.

...

recommendations finished on 369000/372991 queries. users per second: 315326

recommendations finished on 370000/372991 queries. users per second: 315217

recommendations finished on 371000/372991 queries. users per second: 314842

recommendations finished on 372000/372991 queries. users per second: 314201

+--------+--------+-------+------+
| userId | itemId | score | rank |
+--------+--------+-------+------+
|   2    | 43979  |  0.0  |  1   |
|   2    |  5132  |  0.0  |  2   |
|   2    | 13757  |  0.0  |  3   |
|   2    | 39090  |  0.0  |  4   |
|   2    |  2227  |  0.0  |  5   |
|   2    | 19597  |  0.0  |  6   |
|   2    |  6033  |  0.0  |  7   |
|   2    |  5704  |  0.0  |  8   |
|   2    |  8056  |  0.0  |  9   |
|   2    | 35380  |  0.0  |  10  |
|   2    | 43979  |  0.0  |  1   |
|   2    |  5132  |  0.0  |  2   |
|   2    | 13757  |  0.0  |  3   |
|   2    | 39090  |  0.0  |  4   |
|   2    |  2227  |  0.0  |  5   |
|   2    | 19597  |  0.0  |  6   |
|   2    |  6033  |  0.0  |  7   |
|   2    |  5704  |  0.0  |  8   |
|   2    |  8056  |  0.0  |  9   |
|   2    | 35380  |  0.0  |  10  |
|   2    | 43979  |  0.0  |  1   |
|   2    |  5132  |  0.0  |  2   |
|   2    | 13757  |  0.0  |  3   |
|   2    | 39090  |  0.0  |  4   |
|   2    |  2227  |  0.0  |  5   |
|   2    | 19597  | 